In [6]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import folium
import geopy.distance
import streamlit as st
from datetime import datetime
from PIL import Image
import inflection

In [7]:
#Funções iniciais

def code_cleaning(df):

    #removendo linhas vazias
    df = df.dropna()
    
    #removendo a coluna 'Switch to order menu' (todos os valores são 0 e não há descrição da coluna)
    df = df.drop(['Switch to order menu'], axis=1)
    
    #removendo linhas duplicadas
    df = df.drop_duplicates().reset_index(drop=True)
    
    #categorizando os restaurantes somente pela primeira categoria informada
    df['Cuisines'] = df['Cuisines'].apply(lambda x: x.split(',')[0])

    return df

#Preenchimento do nome dos países
country_dict = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}

def country_name(country_id):
    return country_dict[country_id]

#Criação do Tipo de Categoria de Comida
def create_price_type(price_range):
    if price_range == 1:
        return 'cheap'
    elif price_range == 2:
        return 'normal'
    elif price_range == 3:
        return 'expensive'
    else:
        return 'gourmet'

#Criação do nome das Cores
color_dict = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

def color_name(color_code):
    return color_dict[color_code]

#Renomear as colunas do DataFrame
def rename_columns(df):
    df = df.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

#Criando nova coluna com valores da 'average_cost_for_two' convertidos para BRL
currency_to_BRL = {
    'Botswana Pula(P)': 0.36,
    'Brazilian Real(R$)': 1,
    'Dollar($)': 4.92,
    'Emirati Diram(AED)': 1.34, 
    'Indian Rupees(Rs.)': 0.059,
    'Indonesian Rupiah(IDR)': 0.00032, 
    'NewZealand($)': 3.03, 
    'Pounds(£)': 6.27,
    'Qatari Rial(QR)': 1.35,
    'Rand(R)': 0.26,
    'Sri Lankan Rupee(LKR)': 0.016,
    'Turkish Lira(TL)': 0.19
}

def convert_to_BRL(average_cost_for_two):
    return currency_to_BRL[average_cost_for_two]

In [12]:
#carregando dataset
# csv_path = r"C:\Users\rodol\Documents\Comunidade DS\repos\ftc\ciclo8_projeto_do_aluno\files\dataset\zomato.csv"
csv_path = '../dataset/zomato.csv'

df = pd.read_csv(csv_path)

df = code_cleaning(df)

df['Country Name'] = df['Country Code'].map(country_name)

df['Price Category'] = df['Price range'].map(create_price_type)

df['Rating Color Name'] = df['Rating color'].map(color_name)

# df['currency_conversion_BRL'] = df['currency'].map(conversao_para_real)
df['average_cost_for_two_brl'] = df['Average Cost for two'] * df['Currency'].map(convert_to_BRL)

df = rename_columns(df)

#removendo a linha 356, pois o average_cost_for_two_brl é um outlier (mais de R$123mi para duas pessoas)
df = df.drop([356,0]).reset_index(drop=True)

In [13]:
df.head().T

,0,1,2,3,4
restaurant_id,6314542,6301293,6315689,6304833,18409457
restaurant_name,Blackbird,Banapple,Bad Bird,Manam,Soban K-Town Grill
country_code,162,162,162,162,162
city,Makati City,Makati City,Makati City,Makati City,Makati City
address,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Hole In The Wall, Floor 4, Century City Mall, ...","Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Level 3, Greenbelt 3, Ayala Center, Greenbelt,..."
locality,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Century City Mall, Poblacion, Makati City","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 3, San Lorenzo, Makati City"
locality_verbose,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Century City Mall, Poblacion, Makati City, Mak...","Greenbelt 2, San Lorenzo, Makati City, Makati ...","Greenbelt 3, San Lorenzo, Makati City, Makati ..."
longitude,121.024562,121.023171,121.027708,121.02038,121.021388
latitude,14.556042,14.556196,14.565899,14.552351,14.552248
cuisines,European,Filipino,American,Filipino,Korean


# Análise Geral

## 1. Quantos restaurantes únicos estão registrados?

In [21]:
df['restaurant_id'].nunique()

6927

## 2. Quantos países únicos estão registrados?

In [22]:
df['country_name'].nunique()

15

## 3. Quantas cidades únicas estão registradas?

In [23]:
df['city'].nunique()

124

## 4. Qual o total de avaliações feitas?

In [24]:
df['votes'].sum()

4193711

## 5. Qual o total de tipos de culinária registrados?

In [25]:
df['cuisines'].nunique()

165

# Análise Países

## 1. Qual o nome do país que possui mais cidades registradas?

In [26]:
cols = ['country_name','city']
df[cols].groupby('country_name').nunique().sort_values('city', ascending=False).reset_index().head(1)

,country_name,city
0,India,49


## 2. Qual o nome do país que possui mais restaurantes registrados?

In [27]:
cols = ['country_name','restaurant_name']
df[cols].groupby('country_name').nunique().sort_values('restaurant_name', ascending=False).reset_index().head(1)

,country_name,restaurant_name
0,India,2480


## 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?


In [28]:
cols = ['country_name','price_range']
df[df['price_range']==4][cols].groupby('country_name').count().sort_values('price_range', ascending=False).reset_index().head(1)

,country_name,price_range
0,United States of America,415


## 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [29]:
cols = ['country_name','cuisines']
df[cols].groupby('country_name').nunique().sort_values('cuisines', ascending=False).reset_index().head(1)

,country_name,cuisines
0,India,77


## 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [30]:
cols = ['country_name','votes']
df[cols].groupby('country_name').count().sort_values('votes', ascending=False).reset_index().head(1)

,country_name,votes
0,India,3111


## 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [31]:
cols = ['country_name','has_online_delivery']
df[df['has_online_delivery']==1][cols].groupby('country_name').count().sort_values('has_online_delivery', ascending=False).reset_index().head(1)

,country_name,has_online_delivery
0,India,2177


## 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [32]:
cols = ['country_name','has_table_booking']
df[df['has_table_booking']==1][cols].groupby('country_name').count().sort_values('has_table_booking', ascending=False).reset_index().head(1)

,country_name,has_table_booking
0,India,256


## 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [33]:
cols = ['country_name','votes']
df[cols].groupby('country_name').mean().sort_values('votes', ascending=False).reset_index().head(1)

,country_name,votes
0,Indonesia,1112.825


## 9. Qual o nome do país que possui, na média, a maior nota média registrada?

In [34]:
cols = ['country_name','aggregate_rating']
df[cols].groupby('country_name').mean().sort_values('aggregate_rating', ascending=False).reset_index().head(1)

,country_name,aggregate_rating
0,Indonesia,4.60125


## 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [35]:
cols = ['country_name','aggregate_rating']
df[cols].groupby('country_name').mean().sort_values('aggregate_rating', ascending=True).reset_index().head(1)

,country_name,aggregate_rating
0,Brazil,3.321667


## 11. Qual a média de preço de um prato para dois por país?

In [36]:
cols = ['country_name','average_cost_for_two_brl']
df[cols].groupby('country_name').mean().sort_values('average_cost_for_two_brl', ascending=False).reset_index()

,country_name,average_cost_for_two_brl
0,Singapure,695.872500
1,Philippines,442.599494
2,Australia,350.282011
3,England,272.807700
4,United States of America,270.746812
5,Qatar,234.900000
6,United Arab Emirates,205.980333
7,Canada,205.956667
8,New Zeland,188.329079
9,Brazil,138.812500


# Análise Cidades

## 1. Qual o nome da cidade que possui mais restaurantes registrados?

In [37]:
cols = ['city','restaurant_name']
df[cols].groupby('city').nunique().sort_values('restaurant_name', ascending=False).reset_index().head(1)

,city,restaurant_name
0,Singapore,80


## 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [38]:
cols = ['city','aggregate_rating']
df[cols][df['aggregate_rating'] > 4].groupby('city').count().sort_values('aggregate_rating', ascending=False).reset_index().head(1)

,city,aggregate_rating
0,Bangalore,79


## 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [39]:
cols = ['city','aggregate_rating']
df[cols][df['aggregate_rating'] < 2.5].groupby('city').count().sort_values('aggregate_rating', ascending=False).reset_index().head(1)

,city,aggregate_rating
0,Gangtok,33


## 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?

In [40]:
cols = ['city','average_cost_for_two_brl']
df[cols].groupby('city').mean().sort_values('average_cost_for_two_brl', ascending=False).reset_index().head(1)

,city,average_cost_for_two_brl
0,Pasay City,1440.0


## 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [41]:
cols = ['city','cuisines']
df[cols].groupby('city').nunique().sort_values('cuisines', ascending=False).reset_index().head(1)

,city,cuisines
0,Birmingham,32


## 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [42]:
cols = ['city','has_table_booking']
df[df['has_table_booking'] == 1].groupby('city')['has_table_booking'].count().sort_values(ascending=False).reset_index().head(1)

,city,has_table_booking
0,Bangalore,42


## 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?

In [43]:
cols = ['city','is_delivering_now']
df[df['is_delivering_now'] == 1].groupby('city')['is_delivering_now'].count().sort_values(ascending=False).reset_index().head(1)

,city,is_delivering_now
0,Vadodara,48


## 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [44]:
cols = ['city','has_online_delivery']
df[df['has_online_delivery'] == 1].groupby('city')['has_online_delivery'].count().sort_values(ascending=False).reset_index().head(1)

,city,has_online_delivery
0,Bhopal,75


# Análise Tipos de Culinária

## 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [45]:
df[df['cuisines'] == 'Italian'].sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Darshan,4.9


## 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [46]:
df[(df['cuisines'] == 'Italian') & (df['votes'] != 0)].sort_values(['aggregate_rating',
                                                                    'restaurant_id'], ascending=[True,
                                                                                                 True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Ristorantino,0.0


## 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?

In [47]:
df[df['cuisines'] == 'American'].sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Burger & Lobster,4.9


## 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [48]:
df[(df['cuisines'] == 'American') & (df['votes'] != 0)].sort_values(['aggregate_rating',
                                                                    'restaurant_id'], ascending=[True,
                                                                                                 True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Guy Fieri's Kitchen & Bar,2.2


## 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [49]:
df[df['cuisines'] == 'Arabian'].sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Mandi@36,4.7


## 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [50]:
df[(df['cuisines'] == 'Arabian') & (df['votes'] != 0)].sort_values(['aggregate_rating',
                                                                    'restaurant_id'], ascending=[True,
                                                                                                 True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Raful,0.0


## 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [51]:
df[df['cuisines'] == 'Japanese'].sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Sushi Samba,4.9


## 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [52]:
df[(df['cuisines'] == 'Japanese') & (df['votes'] != 0)].sort_values(['aggregate_rating',
                                                                    'restaurant_id'], ascending=[True,
                                                                                                 True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Banzai Sushi,0.0


## 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [53]:
df[df['cuisines'] == 'Home-made'].sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Kanaat Lokantası,4.0


## 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [54]:
df[(df['cuisines'] == 'Home-made') & (df['votes'] != 0)].sort_values(['aggregate_rating',
                                                                    'restaurant_id'], ascending=[True,
                                                                                                 True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,GurMekan Restaurant,3.7


## 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [55]:
cols = ['cuisines','average_cost_for_two_brl']
df[cols].groupby('cuisines').mean().sort_values('average_cost_for_two_brl', ascending=False).reset_index().head(1)

,cuisines,average_cost_for_two_brl
0,New Mexican,885.6


## 12. Qual o tipo de culinária que possui a maior nota média?

In [56]:
cols = ['cuisines','aggregate_rating']
df[cols].groupby('cuisines').mean().sort_values('aggregate_rating', ascending=False).reset_index().head(1)

,cuisines,aggregate_rating
0,Others,4.9


## 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [57]:
cols = ['cuisines','is_delivering_now','has_online_delivery']
df[cols][(df['is_delivering_now'] == 1) & (df['has_online_delivery'] == 1)].groupby('cuisines').count().sort_values('is_delivering_now', ascending=False).reset_index().head(1)

,cuisines,is_delivering_now,has_online_delivery
0,North Indian,317,317


# Análise Restaurantes

## 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [58]:
cols = ['restaurant_name','votes']
df[cols].sort_values('votes', ascending=False).reset_index(drop=True).head(1)

,restaurant_name,votes
0,Bawarchi,41333


## 2. Qual o nome do restaurante com a maior nota média?

In [59]:
cols = ['restaurant_id','restaurant_name','aggregate_rating']
df[cols].sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True]).reset_index(drop=True).head(1)

,restaurant_id,restaurant_name,aggregate_rating
0,7528,Indian Grill Room,4.9


## 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [60]:
cols = ['restaurant_name','average_cost_for_two_brl']
df[cols].sort_values('average_cost_for_two_brl', ascending=False).reset_index(drop=True).head(1)

,restaurant_name,average_cost_for_two_brl
0,Corner House,3714.6


## 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação? (diferentes de 0)

In [61]:
df[(df['cuisines'] == 'Brazilian') & (df['aggregate_rating'] != 0)].sort_values(['aggregate_rating',
                                                                                 'restaurant_id'], ascending=[True,
                                                                                                              True]).reset_index(drop=True).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
0,Café das Estrelas,3.0


## 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [62]:
df[(df['cuisines'] == 'Brazilian') & (df['country_name'] == 'Brazil')].sort_values(['aggregate_rating',
                                                                                    'restaurant_id'], ascending=[False,True]).head(1)[['restaurant_name','aggregate_rating']]

,restaurant_name,aggregate_rating
190,Braseiro da Gávea,4.9


## 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [63]:
media_votos_online_delivery_yes = round(df[df['has_online_delivery'] == 1]['votes'].mean(),1)
media_votos_online_delivery_no = round(df[df['has_online_delivery'] == 0]['votes'].mean(),1)

print(f'Os restaurantes que aceitam pedidos online têm, em média, {media_votos_online_delivery_yes} votos. \nOs que não aceitam pedidos online têm, em média, {media_votos_online_delivery_no} votos. \nPortanto, há relação positiva entre aceitar pedidos online e número de avaliações registradas.')

Os restaurantes que aceitam pedidos online têm, em média, 838.8 votos. 
Os que não aceitam pedidos online têm, em média, 479.5 votos. 
Portanto, há relação positiva entre aceitar pedidos online e número de avaliações registradas.


## 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [64]:
avg_cost_two_booking_yes = round(df[df['has_table_booking'] == 1]['average_cost_for_two_brl'].mean(),2)
avg_cost_two_booking_no = round(df[df['has_table_booking'] == 0]['average_cost_for_two_brl'].mean(),2)

print(f'Os restaurantes que fazem reserva têm um valor médio para duas pessoas de R$ {avg_cost_two_booking_yes}. \nOs que não fazem reserva têm um valor médio para duas pessoas de R$ {avg_cost_two_booking_no}. \nPortanto, a relação entre fazer reservas e valor médio para duas pessoas é pequena.')

Os restaurantes que fazem reserva têm um valor médio para duas pessoas de R$ 175.75. 
Os que não fazem reserva têm um valor médio para duas pessoas de R$ 143.15. 
Portanto, a relação entre fazer reservas e valor médio para duas pessoas é pequena.


## 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?

In [65]:
avg_cost_two_japanese_usa = round(df[(df['cuisines'] == 'Japanese') & (df['country_name'] == 'United States of America')]['average_cost_for_two_brl'].mean(),2)
avg_cost_two_bbq_usa = round(df[(df['cuisines'] == 'BBQ') & (df['country_name'] == 'United States of America')]['average_cost_for_two_brl'].mean(),2)

print(f'O valor médio para duas pessoas nos restaurantes de culinária japonesa dos EUA é R$ {avg_cost_two_japanese_usa}, enquanto o valor médio para dois nas churrascarias dos EUA é de R$ {avg_cost_two_bbq_usa}. \nPortanto, sim, os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas.')

O valor médio para duas pessoas nos restaurantes de culinária japonesa dos EUA é R$ 277.52, enquanto o valor médio para dois nas churrascarias dos EUA é de R$ 195.04. 
Portanto, sim, os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas.
